In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/mrbane10/transformersfromscratch.git

Cloning into 'transformersfromscratch'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 54 (delta 25), reused 41 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 17.72 KiB | 4.43 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
cd /kaggle/working/transformersfromscratch

/kaggle/working/transformersfromscratch


In [35]:
from config import get_config
import os
import wandb
cfg = get_config()
cfg['model_folder'] = '/kaggle/working/weights'
cfg['tokenizer_file'] = '/kaggle/working/tokenizers/tokenizer_{0}.json' 
cfg['batch_size'] = 16 # Conservative for Kaggle
cfg['num_epochs'] = 20
cfg['preload'] = "05"
cfg['seq_len'] = 410

# Create directories
os.makedirs('/kaggle/working/weights', exist_ok=True)
os.makedirs('/kaggle/working/tokenizers', exist_ok=True)

In [22]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
!wandb.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
/bin/bash: -c: line 2: syntax error: unexpected end of file


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
personal_key_for_api = user_secrets.get_secret("wandb_api_key")
!wandb login $personal_key_for_api

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [24]:
from train import train_model

In [26]:
train_model(cfg)

Using device cuda
Max length of source sentence: 291
Max length of target sentence: 406


step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▆▆▆▇▇▇▇█████
train_loss,██▆▆▆▅▅▄▄▄▅▄▃▄▄▄▄▃▃▃▃▃▃▂▂▃▂▂▂▃▁▃▃▂▂▁▂▁▁▂
val/bleu,▁▁▁▁▁▁
val/cer,▃█▂▁▁▃
val/wer,▂█▂▁▁▃
step,13664
train_loss,3.14542
val/bleu,0
val/cer,0.67521
val/wer,1.04


Preloading model /kaggle/working/weights/tmodel_05.pt


Processing epoch 06: 100%|██████████| 1969/1969 [17:57<00:00,  1.83it/s, loss=3.156]
/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
Processing epoch 19: 100%|██████████| 1969/1969 [17:58<00:00,  1.83it/s, loss=1.823]


step,▁▁▁▁▁▂▂▂▂▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█
train_loss,█▇▆▆▅▆▅▅▅▄▄▄▄▃▄▄▅▄▄▄▃▄▃▃▃▄▃▃▃▂▃▄▃▄▂▂▂▂▁▂
val/bleu,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/cer,▇▆▆█▇▁▇▅▆▆▄▄▅▄
val/wer,▅▅▅█▅▁▆▄▅▅▃▃▃▄
step,39380
train_loss,1.82296
val/bleu,0
val/cer,0.50362
val/wer,0.69444


In [ ]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation

In [36]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(cfg, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config,epoch = 19)
state = torch.load(f"/kaggle/working/{model_filename}")
model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 291
Max length of target sentence: 406


<All keys matched successfully>

In [45]:
# Run validation
run_validation(
    model, val_dataloader, tokenizer_src, tokenizer_tgt, 
    config['seq_len'], device, lambda msg: print(msg), 
    0, None, num_examples=10
)

In [46]:
# Define the translate function since it seems to be missing
def translate(text, max_len=None):
    """
    Translate a single text using the loaded model
    """
    if max_len is None:
        max_len = config['seq_len']
    
    # Tokenize input text
    source_tokens = tokenizer_src.encode(text).ids
    
    # Add SOS and EOS tokens
    sos_idx = tokenizer_src.token_to_id('[SOS]')
    eos_idx = tokenizer_src.token_to_id('[EOS]')
    source_tokens = [sos_idx] + source_tokens + [eos_idx]
    
    # Convert to tensor
    source = torch.tensor(source_tokens).unsqueeze(0).to(device)
    source_mask = torch.ones(1, 1, len(source_tokens)).to(device)

 # Use the greedy_decode function from your training script
    from train import greedy_decode
    
    with torch.no_grad():
        result = greedy_decode(
            model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device
        )
    
    # Decode result
    result_tokens = result.detach().cpu().numpy().tolist()
    translated_text = tokenizer_tgt.decode(result_tokens)
    
    return translated_text

In [86]:
# Additional test sentences
test_sentences = [
    "Hello, how are you?",
    "What is your name?",
    "and the listen and obey",
    "This is a great day!",
    "Can you help me with this problem?",
    "and we will test you with difficulties",
    "who is the most high",
    "how has your day been"
]

print(f"\nMultiple Translation Tests:")
for sentence in test_sentences:
    translation = translate(sentence)
    print(f"'{sentence}' -> '{translation}'")


Multiple Translation Tests:
'Hello, how are you?' -> 'خوش ؟'
'What is your name?' -> 'تمہارا نام کیا ہے ؟'
'and the listen and obey' -> 'اور کان لگا کر سنا کرو اور ( یوں ) سنا دیں'
'This is a great day!' -> 'یہ ایک بہت بڑی آفتوں میں سے ایک دن ہے'
'Can you help me with this problem?' -> 'کیا تم مجھ سے کوئی بات میری مدد کر رہے ہو ؟'
'and we will test you with difficulties' -> 'اور ہم تمہیں خوب جدا کرکے رہنے دیں گے'
'who is the most high' -> 'وہ بلند رتبہ والے ( )'
'how has your day been' -> 'آج کے دن کا وقت آگیا ہے'


In [53]:
import torch
import torch.nn as nn
from pathlib import Path
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import torchmetrics
from torchmetrics.text.bleu import BLEUScore

In [75]:
def validate_model_detailed(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, num_examples=10):
    """
    Validate the model and display source, expected, and predicted translations
    Similar to Umar Jamil's validation format
    """
    model.eval()
    count = 0
    
    # For calculating metrics
    all_predicted = []
    all_expected = []
    
    # Console formatting
    console_width = 120
    # print("="*console_width)
    # print(f"{'VALIDATION RESULTS':^{console_width}}")
    # print("="*console_width)
    
    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            
            # Get inputs
            encoder_input = batch["encoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)
            
            # Ensure batch size is 1 for validation
            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"
            
            # Get source and target texts
            source_text = batch['src_txt'][0]
            target_text = batch['tgt_txt'][0]
            
            # Generate prediction using greedy decode
            model_out = greedy_decode(
                model, encoder_input, encoder_mask, 
                tokenizer_src, tokenizer_tgt, max_len, device
            )
            
            # Decode the prediction
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy().tolist())
            
            # Store for metrics calculation
            all_predicted.append(model_out_text)
            all_expected.append(target_text)
            
            # # Display the results
            # print(f"\nSample {count}:")
            # print("-" * console_width)
            
            # # Source sentence
            # print(f"SOURCE ({config['lang_src'].upper()}):")
            # print(f"  {source_text}")
            
            # # Expected translation
            # print(f"EXPECTED ({config['lang_tgt'].upper()}):")
            # print(f"  {target_text}")
            
            # # Model prediction
            # print(f"PREDICTED ({config['lang_tgt'].upper()}):")
            # print(f"  {model_out_text}")
            
            # Quick visual comparison (highlight differences)
            # print("COMPARISON:")
            expected_words = target_text.split()
            predicted_words = model_out_text.split()
            
            # print(f"  Expected length:  {len(expected_words)} words")
            # print(f"  Predicted length: {len(predicted_words)} words")
            
            if count >= num_examples:
                break
    
    # # Calculate and display metrics
    # print(f"{'OVERALL METRICS':^{console_width}}")
    # print("="*console_width)
    
    if len(all_predicted) > 0:
        # Character Error Rate
        cer_metric = torchmetrics.CharErrorRate()
        cer = cer_metric(all_predicted, all_expected).item()
        
        # Word Error Rate  
        wer_metric = torchmetrics.WordErrorRate()
        wer = wer_metric(all_predicted, all_expected).item()
        
        
        print(f"Character Error Rate (CER): {cer:.4f}")
        print(f"Word Error Rate (WER):      {wer:.4f}")
        print(f"Number of samples:          {len(all_predicted)}")
    
    print("="*console_width)
    
    return all_predicted, all_expected

In [76]:
# Run detailed validation
print(f"\nRunning detailed validation...")
predictions, expectations = validate_model_detailed(
    model, val_dataloader, tokenizer_src, tokenizer_tgt, 
    config['seq_len'], device, num_examples=len(val_dataloader)
)


Running detailed validation...
Character Error Rate (CER): 0.3588
Word Error Rate (WER):      0.5303
Number of samples:          3500


In [77]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def quick_bleu_fix(predictions, expectations):
    smoothing = SmoothingFunction()
    
    # Clean and prepare data
    references = [[exp.split()] for exp in expectations]
    candidates = [pred.split() for pred in predictions]
    
    # Remove empty cases
    valid_pairs = [(r, c) for r, c in zip(references, candidates) if len(r[0]) > 0 and len(c) > 0]
    
    if valid_pairs:
        references, candidates = zip(*valid_pairs)
        bleu = corpus_bleu(list(references), list(candidates), smoothing_function=smoothing.method1)
        return bleu
    return 0.0

# Test the fix
corrected_bleu = quick_bleu_fix(predictions, expectations)
print(f"Corrected BLEU Score: {corrected_bleu:.4f}")

Corrected BLEU Score: 0.4391
